## ============ SVMs (MultiClass Image Classification) ==============

#### SVM Read Link: https://www.cse.iitd.ac.in/~parags/teaching/2023/col774/supp_material/svm.pdf

#### Intel Image Classification Dataset
#### Classes: Class 0 (City), Class 1 (Forest), Class 2 (Ice Mountains), Class 3 (Mountains), Class 4 (Ocean), Class 5 (Streets)

### [Using Libsvm]
##### Doc Link: https://www.csie.ntu.edu.tw/%7Ecjlin/libsvm/

### HyperParameters

In [1]:
'''
IMP POINT: Yes, the parameters of the hyperplane (like w) in the high-dimensional feature space must adhere to the relationships defined by the kernel function. The kernel governs both the structure of the feature space and the relationships among the hyperplane parameters.
'''

'\nIMP POINT: Yes, the parameters of the hyperplane (like w) in the high-dimensional feature space must adhere to the relationships defined by the kernel function. The kernel governs both the structure of the feature space and the relationships among the hyperplane parameters.\n'

In [2]:
'''
SVM's cannot be used for multiclass other than using OvO or OvsAll binary classifiers
'''

"\nSVM's cannot be used for multiclass other than using OvO or OvsAll binary classifiers\n"

In [3]:
#####################################    CLASS DEFN    #####################
#classes               = [0,1]
#classes               = [2,3]
#classes               = [0,1,2]
classes               = [0,1,2,3,4,5]
num_classes           = len(classes)
max_num_classes       = 6

#####################################    HYPERPARAMETERS    #####################
resize_dim            = 16
use_normalization     = 1
#error_factor_C       = 1         # we will use default value
use_gaussian_kernel   = 0         # if = 0; then uses linear kernel
#gauss_scaling_factor = 0.001     # we will use default value 

#####################################    ACCURACY DATA    #######################
# BINARY: [0,1] 
# use_gaussian_kernel = 0
# resize_dim = 16                  accuracy (train, valid):   0.90  0.84
# resize_dim = 32                  accuracy (train, valid):   0.97  0.78   
# use_gaussian_kernel = 1
# resize_dim = 16                  accuracy (train, valid):   0.89  0.85
# resize_dim = 32                  accuracy (train, valid):   0.89  0.85   

# MULTICLASS: [0,1, 2] 
# use_gaussian_kernel = 0
# resize_dim = 16                  accuracy (train, valid):   0.83  0.74  
# use_gaussian_kernel = 1
# resize_dim = 16                  accuracy (train, valid):   0.79  0.76    --------->  Improved Here With Gaussian

# MULTICLASS: [0,1, 2, 3, 4, 5] 
# use_gaussian_kernel = 0
# resize_dim = 16                  accuracy (train, valid):   0.62  0.53
# use_gaussian_kernel = 1
# resize_dim = 16                  accuracy (train, valid):   0.57  0.56    --------->  Improved Here With Gaussian   

enable_debug_prints = 0
flatten_dim       = resize_dim*resize_dim*3 
assert (use_gaussian_kernel == 0 or use_gaussian_kernel == 1)

### Testing Data Before and After Resizing

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img=mpimg.imread('dataset_image/train/0/10006.jpg')
plt.axis('off')  # Turn off the axis
imgplot = plt.imshow(img)

In [ ]:
img=mpimg.imread('dataset_image/train/1/10010.jpg')
plt.axis('off')  # Turn off the axis
imgplot = plt.imshow(img)

In [ ]:
img=mpimg.imread('dataset_image/train/2/100.jpg')
plt.axis('off')  # Turn off the axis
imgplot = plt.imshow(img)

In [ ]:
img=mpimg.imread('dataset_image/train/3/10000.jpg')
plt.axis('off')  # Turn off the axis
imgplot = plt.imshow(img)

In [ ]:
img=mpimg.imread('dataset_image/train/4/10016.jpg')
plt.axis('off')  # Turn off the axis
imgplot = plt.imshow(img)

In [ ]:
img=mpimg.imread('dataset_image/train/5/10070.jpg')
plt.axis('off')  # Turn off the axis
imgplot = plt.imshow(img)

In [ ]:
from PIL import Image
img = Image.open("dataset_image/train/1/14621.jpg")
resized_img = img.resize((resize_dim, resize_dim), Image.Resampling.LANCZOS)    # take average of rgb in neighborhood
plt.axis('off')  # Turn off the axis
imgplot = plt.imshow(resized_img)

### Flatten and Unflatten Array

In [ ]:
import numpy as np                
np_array = np.asarray(resized_img, dtype="int32")
#print (np_array)
flat_array = np_array.flatten()
#print (flat_array)

In [ ]:
unflat_array = np.reshape(np.uint32(flat_array), (resize_dim, resize_dim, 3))
plt.axis('off')  # Turn off the axis
imgplot = plt.imshow(unflat_array)

### Loading and Normalizing Training Data: (Classes: 0 to 5)

In [ ]:
import os

def load_image(infilename):
    img = Image.open(infilename)
    img.load()
    resized_img = img.resize((resize_dim, resize_dim), Image.Resampling.LANCZOS)    # take average of rgb in neighborhood
    np_array    = np.asarray(resized_img, dtype="int32")
    flat_array  = np_array.flatten()
    flat_array  = np.reshape(flat_array, (1, len(flat_array)))
    return flat_array

num_train_samples = [0,0,0,0,0,0]       # ith entry stores the number of train sample in class i
num_valid_samples = [0,0,0,0,0,0]       # ith entry stores the number of valid sample in class i
index_train_class = [0,0,0,0,0,0]       # ith entry stores the starting index of class i in train np array 
index_valid_class = [0,0,0,0,0,0]       # ith entry stores the starting index of class i in valid np array
index = 0
x_train     = np.empty([0, resize_dim*resize_dim*3], dtype="int32")
for label in classes:
    index_train_class[label] = index
    dir_path_name = "dataset_image/train/" + str(label) + "/"
    for filename in os.listdir(dir_path_name):
        if filename.endswith(".jpg"): 
            #print (filename)
            filepath   = os.path.join(dir_path_name, filename)
            newrow     = load_image(filepath)
            x_train    = np.vstack([x_train, newrow])
            num_train_samples[label] += 1
            index += 1

index = 0
x_valid = np.empty([0, resize_dim*resize_dim*3], dtype="int32")
for label in classes:
    index_valid_class[label] = index
    dir_path_name = "dataset_image/val/" + str(label) + "/"
    for filename in os.listdir(dir_path_name):
        if filename.endswith(".jpg"): 
            #print (filename)
            filepath   = os.path.join(dir_path_name, filename)
            newrow     = load_image(filepath)
            x_valid    = np.vstack([x_valid, newrow])
            num_valid_samples[label] += 1
            index += 1

In [ ]:
print ("number of training   samples: ", len(x_train))
print ("number of validation samples: ", len(x_valid))
#print (x_train_1[1])

In [ ]:
if use_normalization == 1:
    x_train = x_train/255
    x_valid = x_valid/255
#print (x_train_1[1])

### Libsvm

In [ ]:
from libsvm.svmutil import *

model = [[None] * max_num_classes for _ in range(max_num_classes)]

cnt_num_classifiers = 0
for class_0 in classes:
    for class_1 in classes:
        if (class_1 <= class_0):
            continue
        
        class_0_start_index  = index_train_class[class_0]
        class_0_end_index    = class_0_start_index + num_train_samples[class_0]
        class_1_start_index  = index_train_class[class_1]
        class_1_end_index    = class_1_start_index + num_train_samples[class_1]
        x_train_0            = x_train[class_0_start_index:class_0_end_index]
        x_train_1            = x_train[class_1_start_index:class_1_end_index]
        x_temp               = np.concatenate((x_train_0, x_train_1), axis=0)
        
        x_train_dict = [{i+1: val for i, val in enumerate(row)} for row in x_temp]     # convert to dictionary
        y_train_list = [class_0]*num_train_samples[class_0] + [class_1]*num_train_samples[class_1]
        
        # -g gauss_scaling_factor  and  -c error_factor_C
        if use_gaussian_kernel == 1:
            model[class_0][class_1] = svm_train(y_train_list, x_train_dict, '-t 2')  # -t 2 specifies gaussian kernel
        else:
            model[class_0][class_1] = svm_train(y_train_list, x_train_dict, '-t 0')  # -t 0 specifies linear kernel
        
        print ("SOLVED LP: ", cnt_num_classifiers)
        cnt_num_classifiers += 1

In [ ]:
if enable_debug_prints == 1:
    # Get dual coefficients (alpha * y) 
    coefficients = model[0][1].get_sv_coef()
    #print("Dual Coefficients:", coefficients)
    filt = []
    for item in coefficients:
        if item != (1.0,):
            filt.append(item)
    print (len(filt))
    
    # Get the bias (rho)
    bias = model[0][1].rho.contents.value
    print("Bias (rho):", bias)

In [ ]:
def pred_binary(matrix, class_0, class_1):                  # binary prediction
    pred_ary  = np.empty(len(matrix), dtype="int32")
    x_dict = [{i+1: val for i, val in enumerate(row)} for row in matrix]
    assert (class_0 < class_1)
    p_label, _, _ = svm_predict([0]*len(x_dict), x_dict, model[class_0][class_1], options='-q')   # [0]*len(x_dict) is a dummy placeholder
    return np.array(p_label, dtype="int32")

                     
def pred(matrix):                                           # predict based on maximum votes among one-to-one classifiers        
    pred_ary  = np.empty([0, len(matrix)], dtype="int32")
    for class_0 in classes:
        for class_1 in classes:
            if (class_1 <= class_0):
                continue
            bin_pred     =  pred_binary(matrix, class_0, class_1)
            pred_ary     =  np.vstack([pred_ary, bin_pred])
            
    pred_ary = np.transpose(pred_ary)
    max_vote_ary  = np.empty(len(matrix), dtype="int32")
    for index in range(0, len(max_vote_ary)):
        counts              = np.bincount(pred_ary[index])
        max_vote_ary[index] = np.argmax(counts)
    return max_vote_ary

### Train and Validation Prediction

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_train_true = []
for label in classes:
    y_train_true  +=   [label]*num_train_samples[label]

y_train_pred  = pred(x_train)
print("PR Report         : \n", classification_report(y_train_true, y_train_pred, labels=classes, zero_division=0))
print("Confusion Matrix  : \n", confusion_matrix(y_train_true, y_train_pred))
print("\nTraining Accuracy        : ", accuracy_score(y_train_true, y_train_pred), "\n")

y_valid_true = []
for label in classes:
    y_valid_true  +=   [label]*num_valid_samples[label]

y_valid_pred  = pred(x_valid)
print("PR Report         : \n", classification_report(y_valid_true, y_valid_pred, labels=classes, zero_division=0))
print("Confusion Matrix  : \n", confusion_matrix(y_valid_true, y_valid_pred))
print("\nValidation Accuracy        : ", accuracy_score(y_valid_true, y_valid_pred), "\n")